In [67]:
import pickle

with open("C:/Users/timv/Git/SpaceSalmon/sim/filter_sim_data.pkl", 'rb') as f:
    sim = pickle.load(f)
    
sim

[array([3.90000e-02, 5.80000e-02, 7.20000e-02, ..., 1.99938e+02,
        1.99960e+02, 1.99979e+02]),
 array([642.99677045, 643.00128726, 642.99400727, ..., 637.67240786,
        637.72403166, 637.74749723]),
 array([ 6.47198101e-03,  4.09711687e-02,  8.76758751e-02, ...,
        -6.76301998e+00, -6.59639457e+00, -6.53356969e+00])]

In [28]:
import json
import sys
import matplotlib.pyplot as plt
file = open("15-2-2020-trim.txt")

#this converts heiarchical data into a flattened dict
def flatten(data, prefix=""):
    flat = dict()
    
    if isinstance(data,dict):
        if prefix:
            prefix += "."
        for k,v in data.items():
            flat.update(flatten(v, prefix + str(k)))
        return flat;
    
    if isinstance(data,list):
        if prefix:
            prefix += "."
        i = 0;
        for v in data:
            flat.update(flatten(v, prefix + str(i)))
            i += 1
        return flat
    
    return {prefix:data}



frames = []
for line in file:
    try:
        data = json.loads(line)
        if data["id"] == "sensor":
            try:
                
                flat = flatten(data)
                flat["id"] = 0.0 #Need to handle enumerated states!
                flt = dict()
                
                for key in flat:
                    flt[key] = float(flat[key])
                
                frames += [flt]
                
            except KeyError:
                print('keyError')
                print(line)
    except ValueError:
        print('line not JSON')
file.close()

#convert time to ms
for frame in frames:
    frame["tick"] = frame["tick"]/1000

#these are the keys of the sensor data
keys = list(json_body[0].keys())
#this is the data in a dict of time series
series = {keys[i]: [data[keys[i]] for data in json_body] for i in range(len(keys))}
#this is the data in a time series of dicts


###TIME TO PLOT###
%matplotlib

#Set up three Y axes
fig, ax1 = plt.subplots()
fig.subplots_adjust(right=0.75)
ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax3.spines["right"].set_position(("axes", 1.2))

#Measured RPM (from diff. mode accel)
spin = [(((a['adxl2.a.0']-a['adxl1.a.0'])/0.01)**0.5)*60/2/3.14 for a in frames]

#Common mode accel
accx = [(a["adxl1.a.0"]+a["adxl2.a.0"])/2 for a in frames]
accz = [(a["adxl1.a.2"]+a["adxl2.a.2"])/2 for a in frames]

ax3.plot(series['tick'],spin, color="orange",label='RPM')
ax1.plot(series['tick'],accx,color="green",label='x accel')
ax1.plot(series['tick'],accz,color="red",label='z accel')
ax2.plot(series['tick'],series['bmp2.p'],color="blue",label='pressure')

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
ax3.legend(loc="lower right")



Using matplotlib backend: Qt5Agg
